In [1]:
# initial imports
import pandas as pd
import numpy as np
from path import Path
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [2]:
!pip install psycopg2
!pip3 install mysql-connector-python-rf

In [3]:
# Import SQL Alchemy dependencies
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import session
from sqlalchemy import create_engine, func
import psycopg2
import mysql.connector
from config import db_password

In [4]:
# Connect to pgAdmin with psycopg2
# Configurations
try:
    connection = psycopg2.connect(
        host = 'whats-in-a-bottle.cuvwsdzowgas.us-west-1.rds.amazonaws.com',
        port = 5432,
        user = 'postgres',
        password = db_password,
        database = 'postgres'
        )
except:
    print("I am unable to connect to the database")
# Connection
cursor=connection.cursor()

In [5]:
#Read in Table
cursor.execute("select * from cleaned_data")
#Convert table to pandas dataframe
cleaned_df = "select * from cleaned_data"
#Check DataFrame
wine_df = pd.read_sql(cleaned_df, con=connection)
wine_df = wine_df.apply(pd.to_numeric)
wine_df = wine_df.sort_values(by=['_c0'])
wine_df.head()

,_c0,points,price,country,designation,province,region_1,variety,winery,tokens,...,1,2,3,4,5,6,7,8,9,10
31,0,96.0,235.0,12,20,6,24,2,15,60.0,...,2701.0,16004.0,23236.0,28401.0,33079.0,39068.0,45330.0,46582.0,63420.0,66669.0
32,1,96.0,110.0,11,20,26,26,9,15,51.0,...,11481.0,33140.0,41643.0,46758.0,55709.0,59455.0,69650.0,79653.0,104662.0,106432.0
33,2,96.0,90.0,12,20,6,26,19,15,47.0,...,10077.0,26468.0,32151.0,49402.0,61157.0,64289.0,76272.0,88398.0,99749.0,119775.0
34,3,96.0,65.0,12,22,27,46,12,15,62.0,...,1546.0,4238.0,13171.0,15324.0,16356.0,25381.0,33140.0,46758.0,56257.0,67144.0
35,4,95.0,66.0,4,20,31,26,9,15,66.0,...,1546.0,17161.0,18176.0,31536.0,33140.0,48479.0,49526.0,65848.0,88378.0,90392.0


In [6]:
# Drop the id column (_c0) and check the datatypes
wine_df = wine_df.drop(['_c0'], axis=1)
wine_df.dtypes

points             float64
price              float64
country              int64
designation          int64
province             int64
region_1             int64
variety              int64
winery               int64
tokens             float64
filtered_tokens    float64
1                  float64
2                  float64
3                  float64
4                  float64
5                  float64
6                  float64
7                  float64
8                  float64
9                  float64
10                 float64
dtype: object

In [7]:
wine_df.head()

,points,price,country,designation,province,region_1,variety,winery,tokens,filtered_tokens,1,2,3,4,5,6,7,8,9,10
31,96.0,235.0,12,20,6,24,2,15,60.0,36.0,2701.0,16004.0,23236.0,28401.0,33079.0,39068.0,45330.0,46582.0,63420.0,66669.0
32,96.0,110.0,11,20,26,26,9,15,51.0,31.0,11481.0,33140.0,41643.0,46758.0,55709.0,59455.0,69650.0,79653.0,104662.0,106432.0
33,96.0,90.0,12,20,6,26,19,15,47.0,30.0,10077.0,26468.0,32151.0,49402.0,61157.0,64289.0,76272.0,88398.0,99749.0,119775.0
34,96.0,65.0,12,22,27,46,12,15,62.0,43.0,1546.0,4238.0,13171.0,15324.0,16356.0,25381.0,33140.0,46758.0,56257.0,67144.0
35,95.0,66.0,4,20,31,26,9,15,66.0,37.0,1546.0,17161.0,18176.0,31536.0,33140.0,48479.0,49526.0,65848.0,88378.0,90392.0


In [8]:
# find null values
for column in wine_df.columns:
    print(f"Column {column} has {wine_df[column].isnull().sum()} null values")

Column points has 0 null values
Column price has 0 null values
Column country has 0 null values
Column designation has 0 null values
Column province has 0 null values
Column region_1 has 0 null values
Column variety has 0 null values
Column winery has 0 null values
Column tokens has 0 null values
Column filtered_tokens has 0 null values
Column 1 has 0 null values
Column 2 has 0 null values
Column 3 has 0 null values
Column 4 has 0 null values
Column 5 has 0 null values
Column 6 has 0 null values
Column 7 has 0 null values
Column 8 has 0 null values
Column 9 has 0 null values
Column 10 has 0 null values


In [9]:
#Transform features
wine_df["1"] = wine_df["1"] / 100
wine_df["2"] = wine_df["2"] / 100
wine_df["3"] = wine_df["3"] / 100
wine_df["4"] = wine_df["4"] / 100
wine_df["5"] = wine_df["5"] / 100
wine_df["6"] = wine_df["6"] / 100
wine_df["7"] = wine_df["7"] / 100
wine_df["8"] = wine_df["8"] / 100
wine_df["9"] = wine_df["9"] / 100
wine_df["10"] = wine_df["10"] / 100
wine_df.head()

,points,price,country,designation,province,region_1,variety,winery,tokens,filtered_tokens,1,2,3,4,5,6,7,8,9,10
31,96.0,235.0,12,20,6,24,2,15,60.0,36.0,27.01,160.04,232.36,284.01,330.79,390.68,453.30,465.82,634.20,666.69
32,96.0,110.0,11,20,26,26,9,15,51.0,31.0,114.81,331.40,416.43,467.58,557.09,594.55,696.50,796.53,1046.62,1064.32
33,96.0,90.0,12,20,6,26,19,15,47.0,30.0,100.77,264.68,321.51,494.02,611.57,642.89,762.72,883.98,997.49,1197.75
34,96.0,65.0,12,22,27,46,12,15,62.0,43.0,15.46,42.38,131.71,153.24,163.56,253.81,331.40,467.58,562.57,671.44
35,95.0,66.0,4,20,31,26,9,15,66.0,37.0,15.46,171.61,181.76,315.36,331.40,484.79,495.26,658.48,883.78,903.92


In [10]:
wine_df.insert(1, "points_ranked", '')
wine_df.insert(3, "price_ranked", '')
wine_df.head()

,points,points_ranked,price,price_ranked,country,designation,province,region_1,variety,winery,...,1,2,3,4,5,6,7,8,9,10
31,96.0,,235.0,,12,20,6,24,2,15,...,27.01,160.04,232.36,284.01,330.79,390.68,453.30,465.82,634.20,666.69
32,96.0,,110.0,,11,20,26,26,9,15,...,114.81,331.40,416.43,467.58,557.09,594.55,696.50,796.53,1046.62,1064.32
33,96.0,,90.0,,12,20,6,26,19,15,...,100.77,264.68,321.51,494.02,611.57,642.89,762.72,883.98,997.49,1197.75
34,96.0,,65.0,,12,22,27,46,12,15,...,15.46,42.38,131.71,153.24,163.56,253.81,331.40,467.58,562.57,671.44
35,95.0,,66.0,,4,20,31,26,9,15,...,15.46,171.61,181.76,315.36,331.40,484.79,495.26,658.48,883.78,903.92


In [11]:
# points statistics
wine_df['points'].describe()

count    137167.000000
mean         87.789279
std           3.220341
min          80.000000
25%          86.000000
50%          88.000000
75%          90.000000
max         100.000000
Name: points, dtype: float64

In [12]:
# price statistics
wine_df['price'].describe()

count    137167.000000
mean         33.133531
std          36.328154
min           4.000000
25%          16.000000
50%          24.000000
75%          40.000000
max        2300.000000
Name: price, dtype: float64

In [13]:
# excellent = 1 = 100-90
# good = 2 = 89.9 - 88
# average = 3 = 87.9 - 86
# ok = 4 = 85.9 - 80

wine_df.loc[(wine_df['points'] >= 90),'points_ranked'] = 'Excellent'
wine_df.loc[(wine_df['points'] <= 89.9) & (wine_df['points'] >= 88),'points_ranked'] = 'Good'
wine_df.loc[(wine_df['points'] <= 87.9) & (wine_df['points'] >= 86),'points_ranked'] = 'Average'
wine_df.loc[(wine_df['points'] < 85.9),'points_ranked'] = 'ok'
points_ranked_count = wine_df['points_ranked'].value_counts()
print(points_ranked_count)

Excellent    42438
ok           33776
Average      33370
Good         27583
Name: points_ranked, dtype: int64


In [14]:
# top_shelf = 1 = 2300 - 40
# high = 2 = 39.9 - 24
# med = 3 = 23.9 - 16
# low = 4 = 15.9 - 4

wine_df.loc[(wine_df['price'] >= 40),'price_ranked'] = 'Top Shelf'
wine_df.loc[(wine_df['price'] <= 39.9) & (wine_df['price'] >= 24),'price_ranked'] = 'High'
wine_df.loc[(wine_df['price'] <= 23.9) & (wine_df['price'] >= 16),'price_ranked'] = 'Med'
wine_df.loc[(wine_df['price'] < 15.9),'price_ranked'] = 'Low'
price_ranked_count = wine_df['price_ranked'].value_counts()
print(price_ranked_count)

High         36105
Top Shelf    35321
Low          33871
Med          31870
Name: price_ranked, dtype: int64


In [15]:
wine_df.tail(20)

,points,points_ranked,price,price_ranked,country,designation,province,region_1,variety,winery,...,1,2,3,4,5,6,7,8,9,10
102817,90.0,Excellent,65.0,Top Shelf,4,20,5,26,12,15,...,131.71,145.71,167.04,218.23,259.64,523.51,648.44,706.22,823.72,850.80
102818,89.0,Good,52.0,Top Shelf,4,20,5,26,12,15,...,4.77,24.37,258.82,285.11,336.28,341.21,362.25,424.25,582.33,648.45
102819,89.0,Good,38.0,High,4,17,5,26,12,15,...,94.92,140.79,334.37,406.28,452.94,604.17,720.83,734.89,767.64,777.51
102820,87.0,Average,37.0,High,4,20,5,26,12,15,...,230.71,258.82,515.55,784.95,997.60,998.92,1071.74,1092.08,1134.32,1180.98
102821,94.0,Excellent,30.0,High,4,17,32,26,15,15,...,57.65,110.76,156.82,209.00,285.11,355.13,363.31,413.59,467.58,661.74
102822,94.0,Excellent,25.0,High,12,20,6,3,9,15,...,38.90,125.31,151.32,326.69,362.96,443.36,467.58,512.47,589.47,617.15
102823,93.0,Excellent,30.0,High,12,20,6,26,25,15,...,13.54,122.52,192.12,243.46,283.28,319.03,333.99,349.96,431.57,467.58
102824,93.0,Excellent,65.0,Top Shelf,12,20,6,24,3,15,...,55.61,156.82,295.11,318.39,696.50,764.89,1196.42,1196.67,1261.94,1342.72
102825,92.0,Excellent,30.0,High,4,20,12,11,3,15,...,6.14,26.89,89.35,114.57,230.71,248.46,460.95,622.08,895.83,1092.08
102826,92.0,Excellent,38.0,High,4,20,12,11,3,15,...,18.92,82.97,167.49,190.94,397.04,549.39,557.09,679.12,782.69,865.77


In [16]:
# Define the features set.
X = wine_df.copy()
X = X.drop("points", axis=1)
X = X.drop("points_ranked", axis=1)
X = X.drop("price_ranked", axis=1)
X.head()

,price,country,designation,province,region_1,variety,winery,tokens,filtered_tokens,1,2,3,4,5,6,7,8,9,10
31,235.0,12,20,6,24,2,15,60.0,36.0,27.01,160.04,232.36,284.01,330.79,390.68,453.30,465.82,634.20,666.69
32,110.0,11,20,26,26,9,15,51.0,31.0,114.81,331.40,416.43,467.58,557.09,594.55,696.50,796.53,1046.62,1064.32
33,90.0,12,20,6,26,19,15,47.0,30.0,100.77,264.68,321.51,494.02,611.57,642.89,762.72,883.98,997.49,1197.75
34,65.0,12,22,27,46,12,15,62.0,43.0,15.46,42.38,131.71,153.24,163.56,253.81,331.40,467.58,562.57,671.44
35,66.0,4,20,31,26,9,15,66.0,37.0,15.46,171.61,181.76,315.36,331.40,484.79,495.26,658.48,883.78,903.92


In [17]:
# Define the target set.
y = wine_df["points_ranked"].values
y[:5]

array(['Excellent', 'Excellent', 'Excellent', 'Excellent', 'Excellent'],
      dtype=object)

In [18]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, train_size=0.80)

In [19]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(109733, 19)
(27434, 19)
(109733,)
(27434,)


In [20]:
# Creating StandardScaler instance
scaler = StandardScaler()

In [21]:
# Fitting Standard Scaller
X_scaler = scaler.fit(X_train)

In [22]:
# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Fitting the Decision Tree Model

In [23]:
#y_train = y_train.astype('int')
y_train

array(['Good', 'Excellent', 'Excellent', ..., 'ok', 'Average', 'ok'],
      dtype=object)

In [24]:
# Creating the decision tree classifier instance
model = tree.DecisionTreeClassifier()

In [25]:
# Fitting the model
model = model.fit(X_train_scaled, y_train)

In [26]:
print(model)

DecisionTreeClassifier()


### Making Predictions Using the Tree Model

In [27]:
# Making predictions using the testing data
predictions = model.predict(X_test_scaled)

### Model Evaluation

In [28]:
#y_test = y_test.astype('int')
y_test

array(['Average', 'Excellent', 'Excellent', ..., 'Excellent', 'Average',
       'Average'], dtype=object)

In [29]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual Average", "Actual Excellent", "Actual Good", "Actual Ok"], columns=["Predicted Average", "Predicted Excellent", "Predicted Good", "Predicted Ok"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

In [30]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted Average,Predicted Excellent,Predicted Good,Predicted Ok
Actual Average,4767,467,636,833
Actual Excellent,642,6679,873,304
Actual Good,633,818,3516,436
Actual Ok,812,238,403,5377


Accuracy Score : 0.7413793103448276
Classification Report
              precision    recall  f1-score   support

     Average       0.70      0.71      0.70      6703
   Excellent       0.81      0.79      0.80      8498
        Good       0.65      0.65      0.65      5403
          ok       0.77      0.79      0.78      6830

    accuracy                           0.74     27434
   macro avg       0.73      0.73      0.73     27434
weighted avg       0.74      0.74      0.74     27434



In [31]:
# List the features sorted in order by feature importance

features = model.feature_importances_
headings = X.columns

print("Feature Importances: Wine trends by Points")
sorted(zip(features, headings), reverse=True)

Feature Importances: Wine trends by Points


[(0.16615464748322986, 'price'),
 (0.10441731874555976, 'filtered_tokens'),
 (0.06141430313590222, '1'),
 (0.05841736548051713, '2'),
 (0.0545746031553461, '10'),
 (0.05233300339881769, '3'),
 (0.05123676470431449, '4'),
 (0.05041524174246094, '8'),
 (0.04987094175173274, 'tokens'),
 (0.049455167819101364, '5'),
 (0.04886485556792733, '6'),
 (0.04855770988259609, '7'),
 (0.04855694456318954, '9'),
 (0.03981372452720531, 'variety'),
 (0.03576012615005733, 'country'),
 (0.03182351953523371, 'province'),
 (0.025687765226196074, 'region_1'),
 (0.01908233766302559, 'designation'),
 (0.003563659467586745, 'winery')]